Блок 5 Подсчёт LTV

In [254]:
# Преамбула
from functools import reduce
from typing import Callable, Any
import os
import sys

# подгрузка python скрипт в notebook/
# Если поменть значение в скрипте, то надо перезагрузить весь notebook.
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from mods.defaults import *

In [255]:
def lt_calculation(user_stats: dict) -> int:
    """Функция возвращает lifetime  для пользователя."""
    user_dates = sorted(user_stats.get(OptLTV.DATETIME))
    return (user_dates[-1] - user_dates[0]).days

In [256]:
def ltv_calculation(user_stats: dict):
    """ Фукцния возращает общую сумму платежей .
        Сложное условие : требуется исключать первый платеж, если он был более NUM 
        Решение : 
        paid = user_stats.get(OptLTV.PAID).copy() копируем чтобы не изменить оригинал.
        paid.pop(0) if paid[0] > NUM else paid
        return sum(list(paid))
    """
    return sum(list(user_stats.get(OptLTV.PAID)))

In [257]:
def calc_users(total_stats: dict, process: Callable[..., int]) -> list:
    """
    Функция для просчёта каких-то данных для каждого пользователя. Это фукнция используеться 
    для анализа.
    :param total_stats: общая статистика
    :param process:  фукнция вычеслитель
    :return: Список кортежей ввида  -> ( пользователь, какое-то вычесляемое значение )
    """
    _result = []
    for user_id, stats in total_stats.items():
        _result.append((user_id, process(stats)))
    return _result

In [258]:
def lt_analyze(total_stats: dict, 
               proc_lt_days=lambda x, y: x + y[1],
               proc_lt_users=lambda x, y: x + 1 if y[1] > 0 else x,
               days=90,
               ) -> dict:
    """
    LT анализ для данной выборке. 
    :param total_stats: выборка данных
    :param proc_lt_days: вычеслитель всего дней
    :param proc_lt_users: вычеслитель всего пользователей 
    :return: результат анализа
    """
    lt_stats = calc_users(total_stats, lt_calculation)
    lt_days = reduce(proc_lt_days, lt_stats, 0)
    lt_users = reduce(proc_lt_users, lt_stats, 0)
    # Проверка сложного условия :
    # Пользователей, у которых между платежами более Days, считать за двух
    # дата первой покупоки.
    first_date = lt_stats[0][1]
    for user, date in lt_stats:
        if date - first_date > days:
            lt_users = +1 
    #
    lifetime = lt_days / lt_users
    return {
        "LT_DAYS": lt_days,
        "LT_USERS": lt_users,
        "LifeTime": lifetime
    }

In [259]:
def ltv_analyze(total_stats: dict) -> dict:
    # Суммароное велечина всех покупок
    ltv_stats = calc_users(total_stats, ltv_calculation)
    ltv_paid = reduce(lambda x, y: x + y[1], ltv_stats, 0)
    ltv_users = reduce(lambda x, y: x + 1, ltv_stats, 0)
    ltv_mid = ltv_paid/ltv_users
    return {
        "LTV_PAID": ltv_paid,
        "LTV_USERS": ltv_users,
        "LTV": ltv_mid,
    }

In [260]:
def collect_dates(total_stats: dict, args) -> dict:
    _user_id, _datetime, _amount_paid = args
    _total_stats = total_stats.copy()
    _stats_user = _total_stats.get(_user_id, {
            OptLTV.PAID: [],
            OptLTV.DATETIME: [],
        })
    paids = _stats_user.get(OptLTV.PAID, [])
    paids.append(_amount_paid)
    dates = _stats_user.get(OptLTV.DATETIME, [])
    dates.append(_datetime)
    _total_stats.update(
        {_user_id: {
            OptLTV.PAID: paids,
            OptLTV.DATETIME: dates,
        }
        })
    return _total_stats

In [261]:
def condition(line: list) -> tuple:
    """Фукнция выделяет три параметра дата платежа, чек платежа, номер пользователя."""
    _datetime = datetime.strptime(line[NameHeaders.DATE], date_format)
    _user_id = line[NameHeaders.USER_ID]
    _amount_paid = conv_float(line[NameHeaders.AMOUNT_PAID])
    return _user_id, _datetime, _amount_paid

In [262]:
# проверка
condition([
    '40177',
    '05.10.2016 3:23', 
    '1041', 
    '0,00337963', 
    'email', 
    'promo', 
    '0', 
    '6128', 
    '13,20'])

('1041', datetime.datetime(2016, 10, 5, 3, 23), 13.2)

In [263]:
def proc_stats(
        filename: str, 
        collect_stats: Callable[[dict, Any], dict], 
        condition: Callable[..., tuple],
        skip_line=False,
        **kwargs
) -> dict:
    """
    функция обходит таблицу и делает по каждой строку расчёты. Затем объединяет результат.
    
    :param filename: путь до файла с данными
    :param collect_stats: фукцния коллеционирования обработанных данных
    :param condition: функция обработки данных
    :param skip_line: Если нужно пропустить первую строку, то следует выставить параметр как True
    :return: общая выборка
    """
    
    total_stats: dict = {}
    with open(os.path.abspath(filename), "r") as file:
        if skip_line is True:
            file.readline()
        for line in file:
            line = line.strip().split('\t')
            total_stats = collect_stats(total_stats, condition(line, **kwargs))
    return total_stats

In [264]:
total_stats = proc_stats(
    filename=data_txt, 
    collect_stats=collect_dates, 
    condition=condition)

In [265]:
# print_collection(total_stats)

In [266]:
print_user(total_stats, '1424')

PAID:[13.8, 12.8, 17.2, 16.0, 14.2, 19.6, 22.4, 14.8, 13.0, 13.6, 12.6, 19.2, 20.2, 16.0, 11.6]
DATETIME: ['01.10.2016', '03.10.2016', '04.10.2016', '04.10.2016', '04.10.2016', '05.10.2016', '06.10.2016', '07.10.2016', '07.10.2016', '07.10.2016', '08.10.2016', '08.10.2016', '08.10.2016', '09.10.2016', '10.10.2016']


In [267]:
lt_calculation(total_stats['1424'])

8

In [268]:
ltv_calculation(total_stats['1424'])

236.99999999999997

In [269]:
# анализ для пользователей с ненуливым LT
lt_analyze(total_stats)

{'LT_DAYS': 173, 'LT_USERS': 43, 'LifeTime': 4.023255813953488}

In [270]:
# анализ для пользователей с любым LT
lt_analyze(total_stats, proc_lt_users=lambda x, y: x + 1)

{'LT_DAYS': 173, 'LT_USERS': 124, 'LifeTime': 1.3951612903225807}

In [271]:
ltv_analyze(total_stats)

{'LTV': 41.36935483870969, 'LTV_PAID': 5129.800000000001, 'LTV_USERS': 124}